<a href="https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2006%20-%20Using_Prompt_Templates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Install required packages
!pip install -q langchain==0.3.26 langchain-openai==0.3.23 langchain-core==0.3.66

In [4]:
import os

# os.environ["OPENAI_API_KEY"]="OPENAI_API_KEY"

from google.colab import userdata
os.environ["OPENAI_API_KEY"]= userdata.get('OPENAI_API_KEY')

In [5]:
# Basic Prompt Template Example
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

template = """Answer the question based on the context below. If the
question cannot be answered using the information provided, answer
with "I don't know".
Context: Quantum computing is an emerging field that leverages quantum mechanics to solve complex problems faster than classical computers.
...
Question: {query}
Answer: """

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

# Create the chain for the prompt
chain = prompt_template | llm | StrOutputParser()

# Set the query you want to ask
input_data = {"query": "What is the main advantage of quantum computing over classical computing?"}

# Run the chain to get the AI-generated answer
response = chain.invoke(input_data)

print("Question:", input_data["query"])
print("Answer:", response)

Question: What is the main advantage of quantum computing over classical computing?
Answer: The main advantage of quantum computing over classical computing is its ability to solve complex problems faster by leveraging quantum mechanics.


In [6]:
# Few-Shot Prompt Template Example
from langchain_core.prompts import FewShotPromptTemplate

examples = [
    {"animal": "lion", "habitat": "savanna"},
    {"animal": "polar bear", "habitat": "Arctic ice"},
    {"animal": "elephant", "habitat": "African grasslands"}
]

example_template = """
Animal: {animal}
Habitat: {habitat}
"""

example_prompt = PromptTemplate(
    input_variables=["animal", "habitat"],
    template=example_template
)

dynamic_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Identify the habitat of the given animal",
    suffix="Animal: {input}\nHabitat:",
    input_variables=["input"],
    example_separator="\n\n",
)

# Create the chain for the dynamic_prompt
chain = dynamic_prompt | llm | StrOutputParser()

# Run the chain with input_data
input_data = {"input": "tiger"}
response = chain.invoke(input_data)

print(response)

Animal: tiger  
Habitat: tropical rainforests, grasslands, and mangrove swamps (primarily in Asia)


In [7]:
# Save prompt template
prompt_template.save("awesome_prompt.json")

# Load prompt template
from langchain_core.prompts import load_prompt
loaded_prompt = load_prompt("awesome_prompt.json")

In [8]:
# Advanced Few-Shot with Personality
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Cell 7: Sarcastic AI Assistant Example
examples = [
    {
        "query": "How do I become a better programmer?",
        "answer": "Try talking to a rubber duck; it works wonders."
    }, {
        "query": "Why is the sky blue?",
        "answer": "It's nature's way of preventing eye strain."
    }
]

example_template = """
User: {query}
AI: {answer}
"""

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

prefix = """The following are excerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative and funny responses to users' questions. Here are some
examples:
"""

suffix = """
User: {query}
AI: """

few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

# Create the chain for the few_shot_prompt_template
chain = few_shot_prompt_template | llm | StrOutputParser()

# Run the chain with input_data
input_data = {"query": "How can I learn quantum computing?"}
response = chain.invoke(input_data)

print(response)

Easy! Just build a tiny particle accelerator in your basement, then politely ask electrons to behave. Or, you know, start with some online courses and quantum mechanics basics—less radiation, more brain power.


In [9]:
# Extended examples list
examples = [
    {
        "query": "How do you feel today?",
        "answer": "As an AI, I don't have feelings, but I've got jokes!"
    }, {
        "query": "What is the speed of light?",
        "answer": "Fast enough to make a round trip around Earth 7.5 times in one second!"
    }, {
        "query": "What is a quantum computer?",
        "answer": "A magical box that harnesses the power of subatomic particles to solve complex problems."
    }, {
        "query": "Who invented the telephone?",
        "answer": "Alexander Graham Bell, the original 'ringmaster'."
    }, {
        "query": "What programming language is best for AI development?",
        "answer": "Python, because it's the only snake that won't bite."
    }, {
        "query": "What is the capital of France?",
        "answer": "Paris, the city of love and baguettes."
    }, {
        "query": "What is photosynthesis?",
        "answer": "A plant's way of saying 'I'll turn this sunlight into food. You're welcome, Earth.'"
    }, {
        "query": "What is the tallest mountain on Earth?",
        "answer": "Mount Everest, Earth's most impressive bump."
    }, {
        "query": "What is the most abundant element in the universe?",
        "answer": "Hydrogen, the basic building block of cosmic smoothies."
    }, {
        "query": "What is the largest mammal on Earth?",
        "answer": "The blue whale, the original heavyweight champion of the world."
    }, {
        "query": "What is the fastest land animal?",
        "answer": "The cheetah, the ultimate sprinter of the animal kingdom."
    }, {
        "query": "What is the square root of 144?",
        "answer": "12, the number of eggs you need for a really big omelette."
    }, {
        "query": "What is the average temperature on Mars?",
        "answer": "Cold enough to make a Martian wish for a sweater and a hot cocoa."
    }
]


In [10]:
from langchain_core.example_selectors import LengthBasedExampleSelector

# Use the correct example_prompt from Cell 7 (query/answer format)
query_answer_example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template="""
User: {query}
AI: {answer}
"""
)

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=query_answer_example_prompt,
    max_length=100
)

In [11]:
# Dynamic prompt template with example selector
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=query_answer_example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
)

In [12]:
# Test the dynamic prompt template
# Create the chain for the dynamic_prompt_template
chain = dynamic_prompt_template | llm | StrOutputParser()

# Run the chain with input_data
input_data = {"query": "Who invented the telephone?"}
response = chain.invoke(input_data)

print(response)

Alexander Graham Bell, the original 'ringmaster' who made sure we could all stop yelling across the room!
